In [ ]:
import numpy as np
import open3d as o3d
from sklearn.cluster import KMeans
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

def load_point_cloud(file_path):
    """加载点云数据"""
    pcd = o3d.io.read_point_cloud(file_path)
    return np.asarray(pcd.points)

def filter_ground_points(points, threshold):
    """根据给定的阈值过滤地面点"""
    return points[points[:, 2] > threshold]

def downsample_points(points, voxel_size=0.1):
    """对点云数据进行下采样"""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd = pcd.voxel_down_sample(voxel_size)
    return np.asarray(pcd.points)

def k_means_clustering(points, n_clusters):
    """执行K-means聚类并返回电塔中心坐标"""
    kmeans = KMeans(n_clusters=n_clusters, n_init='auto')
    kmeans.fit(points[:, :2])
    tower_centers = kmeans.cluster_centers_
    return tower_centers

def segment_single_span_lines(points, tower_centers):
    """根据电塔中心坐标分割单档电力线"""
    distances = np.linalg.norm(points[:, :2][:, np.newaxis, :] - tower_centers, axis=2)
    closest_tower_indices = np.argmin(distances, axis=1)
    span_lines = {i: points[closest_tower_indices == i] for i in range(len(tower_centers))}
    return span_lines

def separate_single_line(points, lradius=3.0):
    """单根电力线分离算法"""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    kd_tree = o3d.geometry.KDTreeFlann(pcd)
    
    line_number = 0
    unsearched_mask = np.ones(len(points), dtype=bool)
    single_lines = []
    
    while np.any(unsearched_mask):
        seed_index = np.where(unsearched_mask)[0][0]
        if seed_index is None:
            break
        
        queue = [seed_index]
        current_line = []
        
        while queue:
            point_index = queue.pop(0)
            if not unsearched_mask[point_index]:
                continue
            
            current_line.append(point_index)
            unsearched_mask[point_index] = False
            
            _, indices, _ = kd_tree.search_radius_vector_3d(pcd.points[point_index], lradius)
            for idx in indices:
                if unsearched_mask[idx]:
                    queue.append(idx)
        
        if current_line:
            single_lines.append(points[current_line])
            line_number += 1
    
    return single_lines

def find_connected_lines(lines, threshold=2.0):
    """找到首尾相连的电力线"""
    connected_groups = []
    used_lines = set()
    
    def is_connected(line1, line2, threshold):
        dist_matrix = np.linalg.norm(line1[:, None, :] - line2, axis=2)
        min_dist = np.min(dist_matrix)
        return min_dist < threshold
    
    for i, line1 in enumerate(lines):
        if i in used_lines:
            continue
        
        group = [i]
        used_lines.add(i)
        
        for j, line2 in enumerate(lines):
            if j in used_lines:
                continue
            if is_connected(line1, line2, threshold):
                group.append(j)
                used_lines.add(j)
        
        connected_groups.append(group)
    
    return connected_groups

def assign_colors_to_lines(lines, connected_groups):
    """为电力线分配颜色，首尾相连的电力线颜色相同"""
    colors = np.zeros((len(lines), 3))
    color_count = 0
    
    for group in connected_groups:
        color = np.random.rand(3)
        for idx in group:
            colors[idx] = color
        color_count += 1
    
    return colors

def visualize_lines_matplotlib(line_segments, colors):
    """用Matplotlib可视化电力线（侧面视图）"""
    fig = plt.figure(figsize=(10, 7))
    
    # 添加3D子图
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    
    # 绘制所有点
    for i, line in enumerate(line_segments):
        ax.scatter(line[:, 0], line[:, 1], line[:, 2], color=colors[i], s=1)

    # ==== 新增部分：设置侧面视角 ====
    ax.view_init(elev=30,   # 仰角设为0度（水平视角）
                azim=30)    # 方位角设为0度（沿X轴正方向观察）

    # 设置坐标轴标签和范围
    ax.set_xlabel('X轴')
    ax.set_ylabel('Y轴')
    ax.set_zlabel('Z轴')
    ax.set_title('电力线段侧面视图')
    
    # 自动调整坐标轴范围
    all_points = np.vstack(line_segments)
    ax.set_xlim([np.min(all_points[:,0]), np.max(all_points[:,0])])
    ax.set_ylim([np.min(all_points[:,1]), np.max(all_points[:,1])])
    ax.set_zlim([np.min(all_points[:,2]), np.max(all_points[:,2])])

    plt.show()

def process_span_line(points, lradius=3.0):
    """并行处理单档电力线分离"""
    return separate_single_line(points, lradius)

# 主程序
if __name__ == "__main__":
    # 加载点云数据
    file_path = r"E:\11.SCI\6待分离\A.pcd"
    points = load_point_cloud(file_path)
    
    # 下采样点云数据
    points = downsample_points(points, voxel_size=0.1)
    
    # 确定分割阈值
    min_height = np.min(points[:, 2])
    filtered_points = filter_ground_points(points, min_height)
    
    # 使用K-means聚类确定电塔中心坐标
    n_towers = 2
    tower_centers = k_means_clustering(filtered_points, n_towers)
    
    # 分割单档电力线
    span_lines = segment_single_span_lines(filtered_points, tower_centers)
    
    # 并行处理单根电力线分离
    lradius = 3.0
    all_single_lines = []
    for line_points in span_lines.values():
        single_lines = Parallel(n_jobs=-1)(delayed(process_span_line)(line_points, lradius) for _ in range(1))
        all_single_lines.extend(single_lines[0])
    
    # 找到首尾相连的电力线
    connected_groups = find_connected_lines(all_single_lines, threshold=2)
    
    # 为电力线分配颜色
    colors = assign_colors_to_lines(all_single_lines, connected_groups)
    
    # 使用Matplotlib可视化结果（显示侧面视图）
    visualize_lines_matplotlib(all_single_lines, colors)